# **How to build RAG application - PDF method**
---
Demo how to create RAG from PDF file 

## **Library Installation**
Install those required libary

In [5]:
%pip install --quiet -U langchain ## LLM libary
%pip install --quiet -U chromadb ## Vector Storage
# %pip install --quiet -U langchain-chroma ## LLM Vector Storage
%pip install --quiet -U pypdf ## Loading PDFs
%pip install --quiet -U pytest ## Unit testing
%pip install --quiet -U langchain-community ## LLM Community Library
%pip install --quiet -U langchain-ollama ## LLM Ollama Library

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Import some libraries

In [6]:
import argparse
import os
import shutil
from IPython.display import display, Markdown
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores.chroma import Chroma
# from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

Define variables

In [7]:
# PDF_PATH = "./data/pdf/Maestro_Policy_Engine_25.40.00_Configuration_Guide.pdf"
# CHROMA_PATH = "./chroma-database/mpe-db"
# PDF_PATH = "./data/pdf/Monopoly Manual 2007.pdf"
# CHROMA_PATH = "./chroma-database/monopoly-db"
PDF_PATH = "./data/pdf/mysql-tutorial-excerpt-8.0-en.a4.pdf"
CHROMA_PATH = "./chroma-database/mysql-db"

## **Loading PDF Data**
Try to load PDF data

In [8]:

loader = PyPDFLoader(PDF_PATH)
pages = loader.load()
# print(pages)

Split the documents

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=80,
    length_function=len,
    is_separator_regex=False
)
chunks = text_splitter.split_documents(pages)

Preparing embedding

In [10]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")

**(OPTIONAL) Clear Databse**
Clear database if required

In [ ]:
IS_DB_CLEARED = False
if IS_DB_CLEARED:
     if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

## **Preparing Chroma**

In [11]:
db = Chroma(
    persist_directory=CHROMA_PATH,
    embedding_function=embeddings
)

/tmp/ipykernel_1120365/2609319021.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


Calculate Page IDs

In [12]:
last_page_id = None
current_chunck_index = 0

# Calculate the page ID
print(f"Processing {len(chunks)} chunks")
for chunk in chunks:
    source = chunk.metadata.get("source")
    page = chunk.metadata.get("page")
    current_page_id = f"{source}:{page}"
    # print(f"=== Processing {current_page_id} ===")

    # if the page ID is the same as the last one, increment the index
    # print(f"Last page ID: {last_page_id} | Current page ID: {current_page_id}")

    if current_page_id == last_page_id:
        current_chunck_index += 1
    else:
        current_chunck_index = 0
    # print(f"Chunk index: {current_chunck_index}")

    # Calculate the chunk ID
    chunk_id = f"{current_page_id}:{current_chunck_index}"
    last_page_id = current_page_id
    # print(f"Chunk ID: {chunk_id}")

    # Add it to the page meta-data
    chunk.metadata["id"] = chunk_id

Processing 158 chunks


Add or update the documents

In [13]:
existing_items = db.get(include=[]) # IDs are always included by default
existing_ids = set(existing_items["ids"])
print(f"Number of existing documents in DB: {len(existing_ids)}")

Number of existing documents in DB: 0


Only add document that don't exist in the DB.

In [14]:
new_chunks = []
for chunk in chunks:
    if chunk.metadata["id"] not in existing_ids:
        new_chunks.append(chunk)

if len(new_chunks) > 0:
    print(f"Adding new documents to DB: {len(new_chunks)}")
    chunks = [chunk.metadata["id"] for chunk in new_chunks]
    db.add_documents(new_chunks, ids=chunks)
    db.persist()
else:
    print("No new documents to add to DB")

Adding new documents to DB: 158


/tmp/ipykernel_1120365/1204274233.py:10: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


## **Query Data**
Preparing to search from DB

In [18]:
# reuse existing db
query_text = "How to get information about databases and table from MySql server?"
PROMPT_TEMPLATE = """
1. If not sure, say "I don't know".
2. Answer the question based only on the following context:

Context: {context}

---

Answer the question based on the above context: {question}
"""
results = db.similarity_search_with_score(query_text, k=5)
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
print(f"Context: \n{context_text}")
print(f"Question: \n{query_text}")
prompt = prompt_template.format(context=context_text, question=query_text)

Context: 
Chapter 5 Getting Information About Databases and Tables
What if you forget the name of a database or table, or what the structure of a given table is (for
example, what its columns are called)? MySQL addresses this problem through several statements
that provide information about the databases and tables it supports.
You have previously seen SHOW DATABASES, which lists the databases managed by the server. To
find out which database is currently selected, use the DATABASE() function:
mysql> SELECT DATABASE();
+------------+
| DATABASE() |
+------------+
| menagerie  |
+------------+
If you have not yet selected any database, the result is NULL.
To find out what tables the default database contains (for example, when you are not sure about the
name of a table), use this statement:

---

operations:
• Create a database
• Create a table
• Load data into the table
• Retrieve data from the table in various ways
• Use multiple tables
The menagerie database is simple (deliberately),

Preparing Model for prompt

In [19]:
# model = Ollama(model="tinyllama")
# model = Ollama(model="tinydolphin")
# model = Ollama(model="deepseek-r1:8b")
model = Ollama(model="deepseek-r1:1.5b")
response_text = model.invoke(prompt)

sources = [doc.metadata.get("id", None) for doc, _score in results]
formatted_response = f"Response:\n\n{response_text}\n\nSources: {sources}"
display(Markdown(formatted_response))

Response:

<think>
Alright, let me try to figure out how to answer this question. The user has provided some context about using MySQL to get information about databases and tables, specifically through the SHOW DATABASES command and the SHOW TABLES statement.

First, I need to understand what exactly the user is asking for. They want to know how to retrieve details about the current database or all databases on the server, as well as the columns of a specific table. 

Looking at the context, I see that MySQL uses functions like DATABASE() and CREATE TABLE with the LIKE format. The SHOW TABLES statement can be used to list all table structures in memory.

So, to answer this question, I should outline both commands: using DATABASE() to get the current database name and using SHOW TABLES or CREATE TABLE to get a table's columns. This way, the user knows the exact steps they need.
</think>

To retrieve information about databases and tables in MySQL:

1. **Retrieve the current database name**:
   Use the `DBAL:: show DATABASE()` function (or its alias DATABASE()) without any parameters to find out which database is currently selected.

2. **Retrieve all table structures**:
   Use `mysql> SHOW TABLES;` or `mysql> CREATE TABLE <table_name> LIKE `<schema>;` to view the column definitions of a specific table.

3. **List columns using `SHOW TABLES`**:
   Execute `mysql> SHOW TABLES;` directly in MySQL to display all tables and their respective columns.

These commands allow you to get essential information about your MySQL databases and tables efficiently.

Sources: ['./data/pdf/mysql-tutorial-excerpt-8.0-en.a4.pdf:32:0', './data/pdf/mysql-tutorial-excerpt-8.0-en.a4.pdf:14:3', './data/pdf/mysql-tutorial-excerpt-8.0-en.a4.pdf:6:1', './data/pdf/mysql-tutorial-excerpt-8.0-en.a4.pdf:6:0', './data/pdf/mysql-tutorial-excerpt-8.0-en.a4.pdf:0:0']